# Install Some Libraries

In [ ]:
!pip -qq install datasets sentencepiece

# Data Preparation

## Download dataset [news-corpus](https://github.com/binhvq/news-corpus) from binhvq

Datasets containing full Vietnamese news until 21/05/2021

We will download partially pre-processed, publicly available this dataset from [Huggingface](https://huggingface.co/datasets/vietgpt/binhvq_news_vi)

In [ ]:
from datasets import load_dataset

HUGGINGFACE_PATH = "tdtunlp/binhvq_news_vi"
NUM_CORE = 2 # Colab Maximum Core

dataset = load_dataset(
    path=HUGGINGFACE_PATH,
    data_files="data/train-00000-of-00009-848cb14e692f7fe1.parquet",
    num_proc=NUM_CORE,
    split="train",
    verification_mode="no_checks")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
dataset

Dataset({
    features: ['text'],
    num_rows: 2151733
})

In [ ]:
dataset[:4], # data looks clean!

({'text': ['Sau đó, doanh nghiệp này điều chỉnh giá đi xuống nhưng vẫn cao hơn hôm trước 40.000 đồng/lượng, ở mức 36,86 triệu đồng/lượng-36,96 triệu đồng/lượng đối với giao dịch lẻ và 36,87 triệu đồng/lượng-36,95 triệu đồng/lượng đối với giao dịch buôn.',
   'Đội hình thi đấu 2 đội: AC Milan: Abbiati, Nesta (Bonera), Zambrotta, Abate, Silva, Van Bommel (Noceniro), Seedorf, Aquilani, Boateng, Ibrahimovic, Robinho (Pato) Barcelona: Valdes, Puyol, Abidal, Fabregas (Pedro), Xavi, Thiago (Dos Santos), Mascherano, Keita, Busquets, Villa (Sanchez), Messi Chính Nghĩa.',
   'Nguyễn Hà My Email: vonga_kachiusa@yahoo.com ĐC: Phòng 11, số nhà 434/7 đường Bình Quới, phường 28, quận Bình Thạnh, TP.HCM ĐT: 0985 485 379 Bài: ĐT Ba Lan và những giấc mơ hiển hiện (http://thethao.vietnamnet.vn/vn/euro/ban-doc/14733/dt-ba-lan-va-nhung-giac-mo-hien-hien.html ) Ngày đăng: 08/06/2012 01:51 5.',
   'ĐỘI HÌNH THI ĐÂÚChelsea: Courtois, Azpilicueta, Luiz, Cahill, Moses (Zouma, 85 ), Kante, Fabregas, Alonso, Pedr

We will select random 100,000 docs to train the tokenizer

In [ ]:
NUM_SAMPLES = 100_000

sub_dataset = dataset.shuffle(seed=42)[:NUM_SAMPLES]

## Converting to text file (.txt)
as the `sentencepiece` library input raw file with line-by-line text.

More details on https://github.com/google/sentencepiece?tab=readme-ov-file#train-sentencepiece-model


In [ ]:
from tqdm import tqdm

OUTPUT_TXT_FILE_PATH = "raw_docs.txt"

with open(OUTPUT_TXT_FILE_PATH, "w") as fOut:
    for sample in tqdm(dataset):
        fOut.write(sample['text'].strip() + "\n")


100%|██████████| 2151733/2151733 [01:22<00:00, 26083.49it/s]


# Training Vietnamese Tokenizer with Sentencepiece

In [ ]:
import sentencepiece as spm

In [ ]:
INPUT_DATA_FILES = [OUTPUT_TXT_FILE_PATH] # support mutitples paths
MODEL_PREFIX= "vietnamese_sp"
VOCAB_SIZE = 20_000

# About 200 seconds
# Llama2 tokenizer training settings
## https://github.com/karpathy/llama2.c/blob/master/doc/train_llama_tokenizer.md
spm.SentencePieceTrainer.Train(
    input=INPUT_DATA_FILES,
    model_prefix=MODEL_PREFIX,
    vocab_size=VOCAB_SIZE,
    self_test_sample_size=0,
    input_format="text",
    num_threads=NUM_CORE,
    split_digits=True,
    allow_whitespace_only_pieces=True,
    byte_fallback=True,
    unk_surface=r" \342\201\207 ",
    model_type='bpe',
    max_sentence_length=999_999_999, # no maximum
    input_sentence_size=200_000_000, # no input sentence
    normalization_rule_name="identity",
    shuffle_input_sentence=True
)

# Output .model and .vocab file

Check file `vietnamese_sp.vocab` to see all of the subwords and their corresponding index

# Merge the trained tokenizers to the original Llama2

In [ ]:
!pip uninstall -y protobuf
!pip install --no-binary=protobuf protobuf

# Click restart sessions

Found existing installation: protobuf 3.20.3
Uninstalling protobuf-3.20.3:
  Successfully uninstalled protobuf-3.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.4/422.4 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for protobuf: filename=protobuf-5.28.0-cp310-cp310-linux_x86_64.whl size=1100615 sha256=fe204d2a9c9e28354c8546d36040f8e14864ca4ff904dd48c4ae45170135ea90
  Stored in directory: /root/.cache/pip/wheels/4b/40/5d/c217f7b0d2b541a79338d11fe750739e3304d5c709629a0338
Successfully built protobuf
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.28.0 which is incompatible.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.28.0 which is i

In [ ]:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"]="python"

from transformers import LlamaTokenizer
from sentencepiece import sentencepiece_model_pb2 as sp_pb2_model
import sentencepiece as spm


In [ ]:
LLAMA_TOKENIZER_DIR = "NousResearch/Llama-2-7b-hf" # Duplicated Meta's version Not require HF token for official meta
VIETNAMESE_SP_MODEL_FILE = "vietnamese_sp.model" # trained file

In [ ]:
# load
llama_tokenizer = LlamaTokenizer.from_pretrained(LLAMA_TOKENIZER_DIR)
vietnamese_sp_model = spm.SentencePieceProcessor()
vietnamese_sp_model.Load(VIETNAMESE_SP_MODEL_FILE)

llama_spm = sp_pb2_model.ModelProto()
llama_spm.ParseFromString(llama_tokenizer.sp_model.serialized_model_proto())
vietnamese_spm = sp_pb2_model.ModelProto()
vietnamese_spm.ParseFromString(vietnamese_sp_model.serialized_model_proto())

# print number of tokens
print(len(llama_tokenizer),len(vietnamese_sp_model))

print(llama_tokenizer.all_special_tokens)
print(llama_tokenizer.all_special_ids)
print(llama_tokenizer.special_tokens_map)

## Add Vietnamse tokens to LLaMA tokenizer
llama_spm_tokens_set=set(p.piece for p in llama_spm.pieces)


print(len(llama_spm_tokens_set))
print(f"Before:{len(llama_spm_tokens_set)}")
for p in vietnamese_spm.pieces:
    piece = p.piece
    if piece not in llama_spm_tokens_set:
        new_p = sp_pb2_model.ModelProto().SentencePiece()
        new_p.piece = piece
        new_p.score = 0
        llama_spm.pieces.append(new_p)
print(f"New model pieces: {len(llama_spm.pieces)}")

## Save
output_sp_dir = 'merged_tokenizer_sp' # save as SP format
output_hf_dir = 'merged_tokenizer_hf' # the path to save Vietnamese-LLaMA tokenizer as HF format
os.makedirs(output_sp_dir,exist_ok=True)
with open(output_sp_dir+'/vietnamese_llama.model', 'wb') as f:
    f.write(llama_spm.SerializeToString())

tokenizer = LlamaTokenizer(vocab_file=output_sp_dir+'/vietnamese_llama.model', legacy=False)

tokenizer.save_pretrained(output_hf_dir)
print(f"Vietnamese-LLaMA tokenizer has been saved to {output_hf_dir}")


32000 20000
['<s>', '</s>', '<unk>']
[1, 2, 0]
{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}
32000
Before:32000
New model pieces: 45937
Vietnamese-LLaMA tokenizer has been saved to merged_tokenizer_hf


# Evaluate new tokenizers

In [ ]:
# Test
llama_tokenizer = LlamaTokenizer.from_pretrained(LLAMA_TOKENIZER_DIR)
vietnamese_llama_tokenizer = LlamaTokenizer.from_pretrained(output_hf_dir)
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)
print(tokenizer.special_tokens_map)

text='''Trái đất tròn quay, mặt trời mọc ngàn dặm xa, đánh thức cuộc sống bình minh. Những cánh diều trắng bay cao trên bầu trời, gió êm đềm thổi qua biển cỏ xanh mơn mởn. Cảm giác hạnh phúc như những bông hoa đua nở trong ánh nắng ấm áp của mùa xuân. Điều quan trọng là ta phải yêu và trân trọng cuộc sống này.
The primary use of LLaMA is research on large language models, including'''
print("Test text:\n",text)
print("---------------")
print(f"Tokenized by LLaMA tokenizer:{llama_tokenizer.tokenize(text)}")
print(f"Tokenized by Vietnamese-LLaMA tokenizer:{vietnamese_llama_tokenizer.tokenize(text)}")
print("---------------")
print(f"Tokenized by LLaMA tokenizer IDs: "  + str(llama_tokenizer(text)["input_ids"]))
print(f"Tokenized by Vietnamese-LLaMA tokenizer IDs: "  + str(vietnamese_llama_tokenizer(text)["input_ids"]))

text='''<unk>

'''
print("Test text:\n",text)
print("---------------")
print(f"Tokenized by LLaMA tokenizer:{llama_tokenizer.tokenize(text)}")
print(f"Tokenized by Vietnamese-LLaMA tokenizer:{vietnamese_llama_tokenizer.tokenize(text)}")
print("---------------")
print(f"Tokenized by LLaMA tokenizer IDs: "  + str(llama_tokenizer(text)["input_ids"]))
print(f"Tokenized by Vietnamese-LLaMA tokenizer IDs: "  + str(vietnamese_llama_tokenizer(text)["input_ids"]))




['<s>', '</s>', '<unk>']
[1, 2, 0]
{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}
Test text:
 Trái đất tròn quay, mặt trời mọc ngàn dặm xa, đánh thức cuộc sống bình minh. Những cánh diều trắng bay cao trên bầu trời, gió êm đềm thổi qua biển cỏ xanh mơn mởn. Cảm giác hạnh phúc như những bông hoa đua nở trong ánh nắng ấm áp của mùa xuân. Điều quan trọng là ta phải yêu và trân trọng cuộc sống này.
The primary use of LLaMA is research on large language models, including
---------------
Tokenized by LLaMA tokenizer:['▁Tr', 'á', 'i', '▁', 'đ', 'ấ', 't', '▁tr', 'ò', 'n', '▁qu', 'ay', ',', '▁m', '<0xE1>', '<0xBA>', '<0xB7>', 't', '▁tr', 'ờ', 'i', '▁m', 'ọ', 'c', '▁ng', 'àn', '▁d', '<0xE1>', '<0xBA>', '<0xB7>', 'm', '▁x', 'a', ',', '▁', 'đ', 'án', 'h', '▁th', 'ứ', 'c', '▁cu', 'ộ', 'c', '▁s', 'ố', 'ng', '▁b', 'ì', 'n', 'h', '▁min', 'h', '.', '▁N', 'h', 'ữ', 'ng', '▁c', 'án', 'h', '▁di', 'ề', 'u', '▁tr', 'ắ', 'ng', '▁bay', '▁ca', 'o', '▁tr', 'ên', '▁b', 'ầ', 'u', '▁tr', 'ờ', 'i',

In [ ]:
# Evaluate

texts = ["Cái cây cổ thụ ở trước cửa nhà đã tồn tại hơn một thế kỷ.",
"Thị trấn nhỏ này nổi tiếng với bữa ăn đường phố ngon và giá cả phải chăng.",
"Người dân đang tụ tập tại sân trường để chào đón đội bóng chiến thắng.",
"Sự hiểu biết và tôn trọng đối với người khác là cơ sở của hòa bình xã hội.",
"Vào mùa đông, tuyết rơi dày đặc và biến cảnh quang trường thành một cảnh trắng xóa.",
"Cuộc sống nông thôn thường được đánh giá cao vì sự bình yên và gần gũi với thiên nhiên.",
"Người dân địa phương nồng nàn mừng xuân bằng những lễ hội truyền thống.",
"Trong tương lai, con người cần phải bảo vệ môi trường để bảo vệ tương lai cho thế hệ sau.",
"Trong bức tranh này, một ngọn núi cao nằm dưới bầu trời xanh.",
"Những đứa trẻ vui đùa trong công viên vào buổi chiều mặt trời.",
"Đêm đã đến, và ngôi làng nhỏ im lặng dưới ánh trăng sáng.",
"Người dân đang tận hưởng một buổi hội họp tại quảng trường trung tâm.",
"Cuộc sống là một cuộc hành trình, hãy tận hưởng từng khoảnh khắc.",
"Thiết kế và nghệ thuật đồ họa thường đòi hỏi sự sáng tạo và tư duy.",
"Mùa thu đến, lá cây bắt đầu chuyển màu và rơi xuống đất.",
"Công việc khó khăn luôn đòi hỏi sự kiên nhẫn và nỗ lực."]

In [ ]:
results = []
for text in texts:
    results.append(len(llama_tokenizer.encode(text)))

# Tokens per sentence
sum(results) / len(results), results

(49.5625, [51, 53, 47, 54, 60, 58, 50, 54, 40, 45, 44, 49, 49, 50, 43, 46])

In [ ]:
# Merges x3 ??

results = []
for text in texts:
    results.append(len(vietnamese_llama_tokenizer.encode(text)))


# Tokens per sentence
sum(results) / len(results), results

(17.6875, [17, 19, 17, 20, 20, 21, 16, 23, 16, 15, 16, 16, 16, 18, 18, 15])